Data Science Mini Project - Initial Air Data Processing

Manage Imports

In [1]:
import pandas as pd
import numpy as np

Process the air quality data spreadsheet

In [2]:
air_data_spreadsheet = pd.read_csv('air-quality-data-continuous.csv', sep=";") #Open the file and separate by ';' 

air_data_spreadsheet[['Date', 'Time']] = air_data_spreadsheet['Date Time'].apply(lambda x: pd.Series(str(x).split("T"))) #Split the Date and Time into 2 columns

air_data_spreadsheet[['Year', 'Month', 'Day']] = air_data_spreadsheet['Date'].apply(lambda x: pd.Series(str(x).split("-"))) #Split the Date into year, month, and day

air_data_spreadsheet[['Record_Time', 'Extra']] = air_data_spreadsheet['Time'].apply(lambda x: pd.Series(str(x).split("+"))) #Split the time to remove the extra timezone.

air_data_spreadsheet[["Hour", "Minute", "Second"]] = air_data_spreadsheet["Record_Time"].apply(lambda x: pd.Series(str(x).split(":"))) #Split the time into hour, minutes, and seconds.

air_data_spreadsheet[["Latitude", "Longitude"]] = air_data_spreadsheet["geo_point_2d"].apply(lambda x: pd.Series(str(x).split(","))) #Split the co-ordinates into longitude and latitude
 
air_data_spreadsheet.head()

,Date Time,NOx,NO2,NO,SiteID,PM10,NVPM10,VPM10,NVPM2.5,PM2.5,...,Year,Month,Day,Record_Time,Extra,Hour,Minute,Second,Latitude,Longitude
0,2019-04-29T23:00:00+00:00,122.25,49.25,47.75,501,30.400,NaN,NaN,NaN,NaN,...,2019,04,29,23:00:00,00:00,23,00,00,51.455269382758324,-2.596648828557916
1,2019-04-30T03:00:00+00:00,42.25,35.25,4.50,501,20.275,NaN,NaN,NaN,NaN,...,2019,04,30,03:00:00,00:00,03,00,00,51.455269382758324,-2.596648828557916
2,2019-05-01T06:00:00+00:00,82.00,48.50,21.75,501,28.125,NaN,NaN,NaN,NaN,...,2019,05,01,06:00:00,00:00,06,00,00,51.455269382758324,-2.596648828557916
3,2019-05-01T18:00:00+00:00,357.25,117.75,156.50,501,27.700,NaN,NaN,NaN,NaN,...,2019,05,01,18:00:00,00:00,18,00,00,51.455269382758324,-2.596648828557916
4,2019-05-02T14:00:00+00:00,194.50,73.25,79.25,501,23.775,NaN,NaN,NaN,NaN,...,2019,05,02,14:00:00,00:00,14,00,00,51.455269382758324,-2.596648828557916


Process the luftdaten spreadsheet (this helps find null PM10 and PM2.5 values). The steps are the same as above.

In [3]:
luftdaten_spreadsheet = pd.read_csv('luftdaten_pm_bristol.csv', sep=";")

luftdaten_spreadsheet[['Date', 'Time']] = luftdaten_spreadsheet['Date Time'].apply(lambda x: pd.Series(str(x).split("T")))

luftdaten_spreadsheet[['Year', 'Month', 'Day']] = luftdaten_spreadsheet['Date'].apply(lambda x: pd.Series(str(x).split("-")))

luftdaten_spreadsheet[['Record_Time', 'Extra']] = luftdaten_spreadsheet['Time'].apply(lambda x: pd.Series(str(x).split("+")))

luftdaten_spreadsheet[["Hour", "Minute", "Second"]] = luftdaten_spreadsheet["Record_Time"].apply(lambda x: pd.Series(str(x).split(":")))

luftdaten_spreadsheet[["Latitude", "Longitude"]] = luftdaten_spreadsheet["Lat Long"].apply(lambda x: pd.Series(str(x).split(",")))
 
luftdaten_spreadsheet.head()

,Sensor ID,Date Time,PM10,PM2.5,Lat Long,Day of Week,Hour of Day,Date,Time,Year,Month,Day,Record_Time,Extra,Hour,Minute,Second,Latitude,Longitude
0,59364,2021-12-31T23:00:00+00:00,20.81,8.32,"51.448, -2.57",5,23,2021-12-31,23:00:00+00:00,2021,12,31,23:00:00,00:00,23,00,00,51.448,-2.57
1,39950,2021-12-31T23:00:00+00:00,16.43,8.33,"51.444, -2.606",5,23,2021-12-31,23:00:00+00:00,2021,12,31,23:00:00,00:00,23,00,00,51.444,-2.606
2,34954,2021-12-31T23:00:00+00:00,1.30,0.99,"51.46, -2.536",5,23,2021-12-31,23:00:00+00:00,2021,12,31,23:00:00,00:00,23,00,00,51.46,-2.536
3,12711,2021-12-31T23:00:00+00:00,7.47,5.10,"51.498, -2.694",5,23,2021-12-31,23:00:00+00:00,2021,12,31,23:00:00,00:00,23,00,00,51.498,-2.694
4,38362,2021-12-31T23:00:00+00:00,26.44,15.45,"51.475834, -2.591864",5,23,2021-12-31,23:00:00+00:00,2021,12,31,23:00:00,00:00,23,00,00,51.475834,-2.591864


Get all the unique locations, and their co-ordinates. Splitting this first allows for less memory use later down the line.

In [4]:
unique_locations = air_data_spreadsheet.groupby("Location").head(1) #Get 1 record for each unique location
unique_locations = unique_locations[['Location', 'Latitude', 'Longitude']] #Get the location name, longitude, and latitude
unique_locations

,Location,Latitude,Longitude
0,Colston Avenue,51.455269382758324,-2.596648828557916
196,Temple Way,51.457949713224494,-2.5839890902854434
1069,Brislington Depot,51.441747180510106,-2.5599558321904605
1472,Parson Street School,51.43267570732794,-2.604956656683554
1861,Wells Road,51.427863888572034,-2.563741533100636
3507,Fishponds Road,51.47804497171107,-2.535230274543903
6439,AURN St Pauls,51.46282941748531,-2.584540816301439
7899,Marlborough Street,51.45914197176008,-2.595432718360466


Process the information to find the closest location to the longitude and latitude of the Luftdaten data set, and combine that information.

In [5]:

Locations = pd.DataFrame({'Location': unique_locations['Location'], 'Longitude': unique_locations['Longitude'].astype(float), 'Latitude': unique_locations['Latitude'].astype(float)}) #Create empty data frame.

Locations = luftdaten_spreadsheet.rename(columns={'Latitude': 'Luft_Latitude', 'Longitude': 'Luft_Longitude'}).merge(Locations, how='cross') #Cross merge to get all possible combinations.

Locations['dist'] = np.sqrt((Locations['Latitude'].astype(float) - Locations['Luft_Latitude'].astype(float))**2 + (Locations['Longitude'].astype(float) - Locations['Luft_Longitude'].astype(float))**2) #Runs some math to find the distance between all combinations

Locations = Locations.groupby(['Latitude', 'Longitude']).filter(lambda x: (x['dist'].min() == x['dist']).any()) #This filters the locations with the shortest distance, to make sure everything is in the correct location.

Locations = Locations[['Location', 'Year', 'Day', 'Month', 'Hour', 'PM10', 'PM2.5']] #Collect the necessary columns.

Locations


,Location,Year,Day,Month,Hour,PM10,PM2.5
0,Colston Avenue,2021,31,12,23,20.810000,8.320000
1,Temple Way,2021,31,12,23,20.810000,8.320000
2,Brislington Depot,2021,31,12,23,20.810000,8.320000
3,Parson Street School,2021,31,12,23,20.810000,8.320000
4,Wells Road,2021,31,12,23,20.810000,8.320000
...,...,...,...,...,...,...,...
4185387,Parson Street School,2019,01,01,01,4.122917,2.214167
4185388,Wells Road,2019,01,01,01,4.122917,2.214167
4185389,Fishponds Road,2019,01,01,01,4.122917,2.214167
4185390,AURN St Pauls,2019,01,01,01,4.122917,2.214167


Merge the data frames together.

In [6]:
joined_data = pd.merge(air_data_spreadsheet, Locations, on=['Year','Day','Month','Hour','Location'], how='outer') #Merges the data frames, ensuring the 5 key components are the same

joined_data = joined_data[["SiteID", "Location", "Year", "Month", "Day", "Hour", "NOx", "NO2", "NO", "PM10_y", "PM2.5_y", "O3"]] #Gets the necessary information with which we will process

joined_data = joined_data.rename(columns={"PM10_y": "PM10", "PM2.5_y": "PM2.5"}) #Renames the columns to make it easier when processing

joined_data.head()

,SiteID,Location,Year,Month,Day,Hour,NOx,NO2,NO,PM10,PM2.5,O3
0,501.0,Colston Avenue,2019,04,29,23,122.25,49.25,47.75,14.898333,9.535417,NaN
1,501.0,Colston Avenue,2019,04,29,23,122.25,49.25,47.75,80.429268,50.960244,NaN
2,501.0,Colston Avenue,2019,04,29,23,122.25,49.25,47.75,21.037083,8.784167,NaN
3,501.0,Colston Avenue,2019,04,29,23,122.25,49.25,47.75,15.057500,8.932917,NaN
4,501.0,Colston Avenue,2019,04,29,23,122.25,49.25,47.75,15.883750,9.662083,NaN


Fill in the NA values with the last known record.

In [15]:
joined_data['PM10'] = joined_data['PM10'].fillna(method='bfill') #Uses fillNA to get all the values.
joined_data['PM2.5'] = joined_data['PM2.5'].fillna(method='bfill')
joined_data['O3'] = joined_data['O3'].fillna(method='bfill')
joined_data = joined_data[joined_data[['NOx', 'NO2', 'NO']].notnull().all(1)] #Filters out unknown NOx, NO2, and NO values.
joined_data

,SiteID,Location,Year,Month,Day,Hour,NOx,NO2,NO,PM10,PM2.5,O3
0,501.0,Colston Avenue,2019,04,29,23,122.25,49.25,47.75,14.898333,9.535417,2.9936
1,501.0,Colston Avenue,2019,04,29,23,122.25,49.25,47.75,80.429268,50.960244,2.9936
2,501.0,Colston Avenue,2019,04,29,23,122.25,49.25,47.75,21.037083,8.784167,2.9936
3,501.0,Colston Avenue,2019,04,29,23,122.25,49.25,47.75,15.057500,8.932917,2.9936
4,501.0,Colston Avenue,2019,04,29,23,122.25,49.25,47.75,15.883750,9.662083,2.9936
...,...,...,...,...,...,...,...,...,...,...,...,...
3873005,501.0,Colston Avenue,2021,12,02,07,328.75,92.50,154.25,4.270000,0.890000,23.2000
3873006,501.0,Colston Avenue,2021,12,02,07,328.75,92.50,154.25,4.040000,1.940000,23.2000
3873007,501.0,Colston Avenue,2021,12,02,07,328.75,92.50,154.25,3.420000,0.800000,23.2000
3873008,501.0,Colston Avenue,2021,12,02,07,328.75,92.50,154.25,3.290000,1.050000,23.2000


Export as CSV so that file can be used within testing.

In [13]:
file = 'processed_air_data.csv'#Sets file name
joined_data.to_csv(file, index=True) #Exports file.